In [1]:
print("Hello World")

Hello World


In [1]:
import duckdb
import findspark
findspark.init()
from pyspark.sql import SparkSession

## Dim Companies

In [32]:
from pyspark.sql.functions import col, count
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import split, element_at

In [2]:
spark = SparkSession.builder.appName("Companies").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/20 03:36:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [14]:
# Connect to the DuckDB database
duck_conn = duckdb.connect('/home/anhcu/Final_ETL_App/etl-app/datawarehouse.duckdb')
query = 'SELECT * FROM dim_companies'

# Fetch data from DuckDB and convert to Pandas DataFrame
pandas_df = duck_conn.execute(query).fetchdf()

duck_conn.close()

In [15]:
# Convert Pandas DataFrame to PySpark DataFrame
spark_df_companies = spark.createDataFrame(pandas_df)

print(spark_df_companies.count())

# Show the Spark DataFrame
spark_df_companies.show()

24/06/20 03:55:47 WARN TaskSetManager: Stage 16 contains a task of very large size (27397 KiB). The maximum recommended task size is 1000 KiB.


256115


24/06/20 03:55:49 WARN TaskSetManager: Stage 19 contains a task of very large size (27397 KiB). The maximum recommended task size is 1000 KiB.


+----------+--------------------+--------------------+--------------+-------------------+--------------------+----------------+--------------------+---------------------+-------------------+---------------------+-----------------------+--------------------+--------------------+
|company_id|        company_name|  company_time_stamp|company_ticket|company_is_delisted|    company_category|company_currency|    company_location|company_exchange_name|company_region_name|company_industry_name|company_industry_sector|company_sic_industry|  company_sic_sector|
+----------+--------------------+--------------------+--------------+-------------------+--------------------+----------------+--------------------+---------------------+-------------------+---------------------+-----------------------+--------------------+--------------------+
|         1|ADMIRALTY BANCORP...|2024-06-12 13:09:...|          AAAB|               true|Domestic Common S...|             USD|      Florida; U.S.A|               

24/06/20 03:55:53 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 19 (TID 19): Attempting to kill Python Worker


In [16]:
# Tạo Window specification
windowSpec = Window.partitionBy("company_ticket").orderBy(spark_df_companies["company_time_stamp"].desc())

# Sử dụng row_number để đánh số thứ tự và giữ lại hàng cuối cùng
df_with_row_num = spark_df_companies.withColumn("row_num", row_number().over(windowSpec))

# Lọc để giữ lại hàng cuối cùng trong mỗi nhóm
spark_df_companies_filtered = df_with_row_num.filter(df_with_row_num.row_num == 1).drop("row_num")

# Hiển thị DataFrame kết quả
print(spark_df_companies_filtered.count())
spark_df_companies_filtered.show()

24/06/20 03:56:22 WARN TaskSetManager: Stage 20 contains a task of very large size (27397 KiB). The maximum recommended task size is 1000 KiB.
24/06/20 03:56:25 WARN TaskSetManager: Stage 26 contains a task of very large size (27397 KiB). The maximum recommended task size is 1000 KiB.


19716


+----------+--------------------+--------------------+--------------+-------------------+--------------------+----------------+--------------------+---------------------+-------------------+---------------------+-----------------------+--------------------+--------------------+
|company_id|        company_name|  company_time_stamp|company_ticket|company_is_delisted|    company_category|company_currency|    company_location|company_exchange_name|company_region_name|company_industry_name|company_industry_sector|company_sic_industry|  company_sic_sector|
+----------+--------------------+--------------------+--------------+-------------------+--------------------+----------------+--------------------+---------------------+-------------------+---------------------+-----------------------+--------------------+--------------------+
|    249567|          ALCOA CORP|2024-06-17 06:01:...|            AA|              false|Domestic Common S...|             USD| Pennsylvania; U.S.A|               

In [30]:
# Thống kê số lượng công ty ở từng sàn giao dịch
print("Thống kê số lượng công ty ở từng sàn giao dịch:")
spark_df_companies_filtered\
    .groupBy("company_exchange_name")\
    .agg(count("*").alias("count"))\
    .withColumnRenamed("company_exchange_name", "Sàn giao dịch")\
    .withColumnRenamed("count", "Số lượng mã niêm yết")\
    .show()

Thống kê số lượng công ty ở từng sàn giao dịch:


24/06/20 04:12:48 WARN TaskSetManager: Stage 74 contains a task of very large size (27397 KiB). The maximum recommended task size is 1000 KiB.


+-------------+--------------------+
|Sàn giao dịch|Số lượng mã niêm yết|
+-------------+--------------------+
|         NYSE|                6552|
|       NASDAQ|               13164|
+-------------+--------------------+



In [37]:
# Thống kê số lượng công ty đã và chưa bị delisted
print("Thống kê số lượng công ty bị hủy niêm yết:")
spark_df_companies_filtered\
    .groupBy("company_is_delisted")\
    .agg(count("*").alias("count"))\
    .withColumnRenamed("company_is_delisted", "Đã bị hủy niêm yết")\
    .withColumnRenamed("count", "Số lượng")\
    .show()

Thống kê số lượng công ty bị hủy niêm yết:


24/06/20 04:20:26 WARN TaskSetManager: Stage 95 contains a task of very large size (27397 KiB). The maximum recommended task size is 1000 KiB.


+------------------+--------+
|Đã bị hủy niêm yết|Số lượng|
+------------------+--------+
|              true|   13200|
|             false|    6516|
+------------------+--------+



In [46]:
# Thống kê số lượng công ty theo khu vực
print("Thống kê số lượng công ty theo quốc gia:")
spark_df_companies_filtered\
    .withColumn("Quóc gia", element_at(split(spark_df_companies_filtered["company_location"], "; "), -1))\
    .drop("company_location")\
    .groupBy("Quóc gia")\
    .agg(count("*").alias("count"))\
    .orderBy(col("count").desc())\
    .withColumnRenamed("count", "Số lượng công ty")\
    .show()

Thống kê số lượng công ty theo quốc gia:


24/06/20 04:27:33 WARN TaskSetManager: Stage 133 contains a task of very large size (27397 KiB). The maximum recommended task size is 1000 KiB.


+--------------+----------------+
|      Quóc gia|Số lượng công ty|
+--------------+----------------+
|         U.S.A|           16554|
|        Canada|             532|
|         China|             527|
|United Kingdom|             273|
|        Israel|             226|
|       Bermuda|             180|
|Cayman Islands|             179|
|     Hong Kong|              97|
|     Singapore|              90|
|       Ireland|              79|
|        Brazil|              75|
|   Netherlands|              60|
|     Australia|              57|
|       Germany|              56|
|    Luxembourg|              51|
|   Switzerland|              45|
|        Mexico|              44|
|         Japan|              44|
|        Greece|              43|
|        France|              41|
+--------------+----------------+
only showing top 20 rows



In [47]:
# Thống kê số lượng công ty theo ngành công nghiệp
print("Thống kê số lượng công ty theo ngành công nghiệp:")
spark_df_companies_filtered\
    .groupBy("company_industry_sector")\
    .agg(count("*").alias("count"))\
    .orderBy(col("count").desc())\
    .withColumnRenamed("company_industry_sector", "Industry")\
    .withColumnRenamed("count", "Số lượng công ty")\
    .show()

Thống kê số lượng công ty theo ngành công nghiệp:


24/06/20 04:27:38 WARN TaskSetManager: Stage 139 contains a task of very large size (27397 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+----------------+
|            Industry|Số lượng công ty|
+--------------------+----------------+
|         Industrials|            3688|
|          Technology|            3394|
|          Healthcare|            2990|
|  Financial Services|            2888|
|   Consumer Cyclical|            1907|
|Communication Ser...|            1119|
|         Real Estate|             986|
|              Energy|             845|
|     Basic Materials|             817|
|  Consumer Defensive|             715|
|           Utilities|             367|
+--------------------+----------------+



In [48]:
# Thống kê số lượng công ty theo ngành công nghiệp theo SIC
print("Thống kê số lượng công ty theo ngành công nghiệp theo SIC:")
spark_df_companies_filtered\
    .groupBy("company_sic_sector")\
    .agg(count("*").alias("count"))\
    .orderBy(col("count").desc())\
    .withColumnRenamed("company_sic_sector", "SIC Industry")\
    .withColumnRenamed("count", "Số lượng công ty")\
    .show()

Thống kê số lượng công ty theo ngành công nghiệp theo SIC:


24/06/20 04:27:43 WARN TaskSetManager: Stage 145 contains a task of very large size (27397 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+----------------+
|        SIC Industry|Số lượng công ty|
+--------------------+----------------+
|       Manufacturing|            6407|
|Finance Insurance...|            5724|
|            Services|            3764|
|Transportation Co...|            1599|
|        Retail Trade|             869|
|              Mining|             705|
|     Wholesale Trade|             418|
|        Construction|             160|
|Agriculture Fores...|              70|
+--------------------+----------------+



In [59]:
# Thống kê số lượng công ty theo loại hình công ty
spark_df_companies_filtered\
    .groupBy("company_category")\
    .agg(count("*").alias("count"))\
    .orderBy(col("count").desc())\
    .withColumnRenamed("company_category", "Category")\
    .withColumnRenamed("count", "Số lượng công ty")\
    .show()

24/06/20 06:27:04 WARN TaskSetManager: Stage 181 contains a task of very large size (27397 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+----------------+
|            Category|Số lượng công ty|
+--------------------+----------------+
|Domestic Common S...|           12196|
|Domestic Common S...|            1810|
|    ADR Common Stock|            1483|
|Domestic Common S...|            1296|
|Domestic Common S...|            1028|
|Domestic Preferre...|            1025|
|Canadian Common S...|             247|
|ADR Common Stock ...|             222|
|ADR Common Stock ...|             179|
|ADR Common Stock ...|             122|
| ADR Preferred Stock|              89|
|Canadian Common S...|              10|
|Canadian Common S...|               4|
|Canadian Preferre...|               3|
|Canadian Common S...|               2|
+--------------------+----------------+



## Fact Candles

In [128]:
import pandas as pd
from datetime import datetime
from pyspark.sql.types import StructType, StructField, IntegerType, LongType, DoubleType, BooleanType, TimestampType
from pyspark.sql.functions import col, count, avg, to_date, when, round
from pyspark.sql.functions import sum as _sum

In [111]:
# Connect to the DuckDB database
duck_conn = duckdb.connect('/home/anhcu/Final_ETL_App/etl-app/datawarehouse.duckdb')
query = 'SELECT * FROM fact_candles'

# Fetch data from DuckDB and convert to Pandas DataFrame
pandas_df = duck_conn.execute(query).fetchdf()

duck_conn.close()

In [112]:
pandas_df

,candle_id,candle_company_id,candle_volume,candle_volume_weighted,candle_open,candle_close,candle_high,candle_low,candle_time_stamp,candle_num_of_trades,candle_is_otc,candles_time_id
0,91400,40924,925828,0.4527,0.5500,0.4000,0.560,0.3901,1718136000000,2943,NaN,1
1,91401,41378,5991,11.0456,11.0200,11.0500,11.070,11.0200,1718136000000,47,NaN,1
2,91402,236408,2720,11.4063,11.3700,11.4000,11.450,11.3700,1718136000000,33,NaN,1
3,91403,236410,129357,1.7232,1.7200,1.7300,1.740,1.7000,1718136000000,806,NaN,1
4,91404,236412,33671,0.3436,0.3525,0.3524,0.358,0.3200,1718136000000,180,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...
36590,127990,256106,159175,11.6804,11.7300,11.6800,11.740,11.6400,1718740800000,2169,NaN,8
36591,127991,256108,847031,2.8864,2.9300,2.9200,2.980,2.7900,1718740800000,4180,NaN,8
36592,127992,256109,196054,21.1173,20.3800,21.3800,21.660,20.3700,1718740800000,2286,NaN,8
36593,127993,256110,446893,3.4015,3.4100,3.5200,3.600,3.3000,1718740800000,1456,NaN,8


In [113]:
# Loại bỏ các cột không cần thiết
pandas_df.drop(columns=['candles_time_id'], inplace=True)

# Chuyển đổi kiểu dữ liệu
pandas_df['candle_is_otc'] = pandas_df['candle_is_otc'].fillna(False).astype(bool)
pandas_df['candle_time_stamp'] = pd.to_datetime(pandas_df['candle_time_stamp'], unit='ms')

pandas_df

/tmp/ipykernel_96936/1880719106.py:6: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  pandas_df['candle_time_stamp'] = pd.to_datetime(pandas_df['candle_time_stamp'], unit='ms')


,candle_id,candle_company_id,candle_volume,candle_volume_weighted,candle_open,candle_close,candle_high,candle_low,candle_time_stamp,candle_num_of_trades,candle_is_otc
0,91400,40924,925828,0.4527,0.5500,0.4000,0.560,0.3901,2024-06-11 19:59:07.264,2943,False
1,91401,41378,5991,11.0456,11.0200,11.0500,11.070,11.0200,2024-06-11 19:59:07.264,47,False
2,91402,236408,2720,11.4063,11.3700,11.4000,11.450,11.3700,2024-06-11 19:59:07.264,33,False
3,91403,236410,129357,1.7232,1.7200,1.7300,1.740,1.7000,2024-06-11 19:59:07.264,806,False
4,91404,236412,33671,0.3436,0.3525,0.3524,0.358,0.3200,2024-06-11 19:59:07.264,180,False
...,...,...,...,...,...,...,...,...,...,...,...
36590,127990,256106,159175,11.6804,11.7300,11.6800,11.740,11.6400,2024-06-18 20:00:44.544,2169,False
36591,127991,256108,847031,2.8864,2.9300,2.9200,2.980,2.7900,2024-06-18 20:00:44.544,4180,False
36592,127992,256109,196054,21.1173,20.3800,21.3800,21.660,20.3700,2024-06-18 20:00:44.544,2286,False
36593,127993,256110,446893,3.4015,3.4100,3.5200,3.600,3.3000,2024-06-18 20:00:44.544,1456,False


In [114]:
# Xác định schema
schema = StructType([
    StructField("candle_id", IntegerType(), True),
    StructField("candle_company_id", IntegerType(), True),
    StructField("candle_volume", LongType(), True),
    StructField("candle_volume_weighted", DoubleType(), True),
    StructField("candle_open", DoubleType(), True),
    StructField("candle_close", DoubleType(), True),
    StructField("candle_high", DoubleType(), True),
    StructField("candle_low", DoubleType(), True),
    StructField("candle_time_stamp", TimestampType(), True),
    StructField("candle_num_of_trades", IntegerType(), True),
    StructField("candle_is_otc", BooleanType(), True)
])

# Chuyển đổi Pandas DataFrame thành PySpark DataFrame với schema
spark_df_candles = spark.createDataFrame(pandas_df, schema=schema)

# Hiển thị một vài hàng của DataFrame
print(spark_df_candles.count())
spark_df_candles.show()

24/06/20 07:37:18 WARN TaskSetManager: Stage 227 contains a task of very large size (1359 KiB). The maximum recommended task size is 1000 KiB.


36595
+---------+-----------------+-------------+----------------------+-----------+------------+-----------+----------+--------------------+--------------------+-------------+
|candle_id|candle_company_id|candle_volume|candle_volume_weighted|candle_open|candle_close|candle_high|candle_low|   candle_time_stamp|candle_num_of_trades|candle_is_otc|
+---------+-----------------+-------------+----------------------+-----------+------------+-----------+----------+--------------------+--------------------+-------------+
|    91400|            40924|       925828|                0.4527|       0.55|         0.4|       0.56|    0.3901|2024-06-11 19:59:...|                2943|        false|
|    91401|            41378|         5991|               11.0456|      11.02|       11.05|      11.07|     11.02|2024-06-11 19:59:...|                  47|        false|
|    91402|           236408|         2720|               11.4063|      11.37|        11.4|      11.45|     11.37|2024-06-11 19:59:...|    

24/06/20 07:37:18 WARN TaskSetManager: Stage 230 contains a task of very large size (1359 KiB). The maximum recommended task size is 1000 KiB.


In [131]:
# Thống kê số lượng công ty theo loại hình công ty
spark_df_candles\
    .withColumn("Ngày giao dịch", to_date(col("candle_time_stamp")))\
    .groupBy("Ngày giao dịch")\
    .agg(
        _sum("candle_volume").alias("Tổng khối lượng cổ phiếu được giao dịch trong ngày"),
        _sum("candle_num_of_trades").alias("Tổng số giao dịch được thực hiện"),
        _sum(when(col("candle_is_otc") == True, 1).otherwise(0)).alias("Số lượng mã OTC")
    )\
    .orderBy(col("Ngày giao dịch"))\
    .show()

24/06/20 08:54:02 WARN TaskSetManager: Stage 321 contains a task of very large size (1359 KiB). The maximum recommended task size is 1000 KiB.


+--------------+--------------------------------------------------+--------------------------------+---------------+
|Ngày giao dịch|Tổng khối lượng cổ phiếu được giao dịch trong ngày|Tổng số giao dịch được thực hiện|Số lượng mã OTC|
+--------------+--------------------------------------------------+--------------------------------+---------------+
|    2024-06-11|                                        8250981825|                        62453553|            340|
|    2024-06-12|                                        9199918379|                        72206567|            367|
|    2024-06-13|                                        8155066574|                        62207544|            356|
|    2024-06-14|                                        8066775751|                        60812069|            361|
|    2024-06-17|                                        9231613973|                        65909662|            371|
|    2024-06-18|                                        93072932

## Mix

In [124]:
# Kết hợp hai bảng dựa trên company_id và candle_company_id
joined_df = spark_df_candles.join(spark_df_companies_filtered, spark_df_candles.candle_company_id == spark_df_companies_filtered.company_id)

# Hiển thị một vài hàng của DataFrame kết hợp
joined_df.show()

24/06/20 08:46:40 WARN TaskSetManager: Stage 252 contains a task of very large size (1359 KiB). The maximum recommended task size is 1000 KiB.
24/06/20 08:46:40 WARN TaskSetManager: Stage 253 contains a task of very large size (27397 KiB). The maximum recommended task size is 1000 KiB.


+---------+-----------------+-------------+----------------------+-----------+------------+-----------+----------+--------------------+--------------------+-------------+----------+--------------------+--------------------+--------------+-------------------+--------------------+----------------+-------------------+---------------------+-------------------+---------------------+-----------------------+--------------------+--------------------+
|candle_id|candle_company_id|candle_volume|candle_volume_weighted|candle_open|candle_close|candle_high|candle_low|   candle_time_stamp|candle_num_of_trades|candle_is_otc|company_id|        company_name|  company_time_stamp|company_ticket|company_is_delisted|    company_category|company_currency|   company_location|company_exchange_name|company_region_name|company_industry_name|company_industry_sector|company_sic_industry|  company_sic_sector|
+---------+-----------------+-------------+----------------------+-----------+------------+-----------+---

In [133]:
# Thống kê khối lượng giao dịch trung bình theo ngành công nghiệp
joined_df\
    .groupBy("company_industry_sector")\
    .agg(round(avg("candle_volume")).alias("Khối lượng trung bình"))\
    .orderBy("Khối lượng trung bình", ascending=False)\
    .withColumnRenamed("company_industry_sector", "Ngành")\
    .show()
    
# Thống kê khối lượng giao dịch trung bình theo ngành công nghiệp theo SIC
joined_df\
    .groupBy("company_sic_sector")\
    .agg(round(avg("candle_volume")).alias("Khối lượng trung bình"))\
    .orderBy("Khối lượng trung bình", ascending=False)\
    .withColumnRenamed("company_sic_sector", "Ngành SIC")\
    .show()

24/06/20 08:56:08 WARN TaskSetManager: Stage 342 contains a task of very large size (1359 KiB). The maximum recommended task size is 1000 KiB.
24/06/20 08:56:08 WARN TaskSetManager: Stage 343 contains a task of very large size (27397 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+---------------------+
|               Ngành|Khối lượng trung bình|
+--------------------+---------------------+
|          Technology|            2067822.0|
|   Consumer Cyclical|            2017198.0|
|              Energy|            1743764.0|
|Communication Ser...|            1664465.0|
|     Basic Materials|            1630475.0|
|  Consumer Defensive|            1403915.0|
|           Utilities|            1295785.0|
|          Healthcare|            1172194.0|
|  Financial Services|            1159888.0|
|         Real Estate|             976227.0|
|         Industrials|             960901.0|
+--------------------+---------------------+



24/06/20 08:56:11 WARN TaskSetManager: Stage 351 contains a task of very large size (1359 KiB). The maximum recommended task size is 1000 KiB.
24/06/20 08:56:11 WARN TaskSetManager: Stage 352 contains a task of very large size (27397 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+---------------------+
|           Ngành SIC|Khối lượng trung bình|
+--------------------+---------------------+
|              Mining|            1964628.0|
|        Retail Trade|            1796498.0|
|Transportation Co...|            1741571.0|
|       Manufacturing|            1708938.0|
|            Services|            1453866.0|
|Finance Insurance...|             777687.0|
|     Wholesale Trade|             658491.0|
|        Construction|             477696.0|
|Agriculture Fores...|             300966.0|
+--------------------+---------------------+



In [139]:
# Top 10 công ty có tổng khối lượng giao dịch nhiều nhất hiện tại
joined_df\
    .groupBy("company_name")\
    .agg(
        _sum("candle_volume").alias("Tổng khối lượng cổ phiếu được giao dịch"),
        _sum("candle_num_of_trades").alias("Tổng số giao dịch được thực hiện")
    )\
    .orderBy("Tổng khối lượng cổ phiếu được giao dịch", ascending=False)\
    .withColumnRenamed("company_name", "Tên công ty")\
    .limit(10)\
    .show()

24/06/20 09:00:37 WARN TaskSetManager: Stage 396 contains a task of very large size (1359 KiB). The maximum recommended task size is 1000 KiB.
24/06/20 09:00:37 WARN TaskSetManager: Stage 397 contains a task of very large size (27397 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+---------------------------------------+--------------------------------+
|         Tên công ty|Tổng khối lượng cổ phiếu được giao dịch|Tổng số giao dịch được thực hiện|
+--------------------+---------------------------------------+--------------------------------+
|         NVIDIA CORP|                             1657885106|                        12087721|
|     MUSTANG BIO INC|                             1428299544|                         1505777|
|NAUTICUS ROBOTICS...|                              885844261|                          882342|
|           APPLE INC|                              703956224|                         7244030|
|CROWN ELECTROKINE...|                              640968519|                          295617|
|       GAMESTOP CORP|                              613487824|                         4668247|
|           TESLA INC|                              532751516|                         6547924|
|ENDRA LIFE SCIENC...|                  